# 特征提取

In [1]:
import sys, os, gc
import numpy as np
import pandas as pd
import pickle
sys.path.append("./global")
from helper import ReadCSV, Timer, ExtractFeature, ORI_TRAIN_NAMES, ORI_TRAIN_DTYPE, ORI_TEST_NAMES, ORI_TEST_DTYPE, AnalysisCSV, OFFLINE, cal_sim
import dist_utils, ngram_utils
base_feature_save_dir = "./stage1/output/"
test_file = "./stage1/input/test.csv"
if OFFLINE:
    base_prefix = "debug_"
    train_file = "./stage1/input/train_last_50w.csv"
    CHUNK_SIZE = 100000
else:
    base_prefix = "online_"
    train_file = "./stage1/input/train_last_1000w.csv"
    CHUNK_SIZE = 1000000
print(OFFLINE)

False


In [12]:
# ! pip install  --index "http://pypi/simple" --trusted-host pypi fuzzywuzzy
# ! pip install  --index "http://pypi/simple" --trusted-host pypi  pyemd
# ! pip install  --index "http://pypi/simple" --trusted-host pypi simhash
! pip install  --index "http://pypi/simple" --trusted-host pypi networkx

## 文本挖掘特征

In [13]:
from fuzzywuzzy import fuzz
feature_save_dir = base_feature_save_dir + "text_mining_feature/"
train_feature_prefix = base_prefix + "trainTextMining"
test_feature_prefix = base_prefix + "testTextMining"
! ls -alh ./stage1/output/text_mining_feature/

total 1.9G
drwxr-xr-x 2 kesci root  4.0K Jun 29 13:47 .
drwxr-xr-x 7 kesci root  4.0K Jun 19 09:08 ..
-rw-r--r-- 1 kesci users 8.6M Jun 26 13:27 debug_trainTextMining_editSim.csv.gz
-rw-r--r-- 1 kesci users 4.6M Jun 26 13:24 debug_trainTextMining_len.csv.gz
-rw-r--r-- 1 kesci users 2.7M Jun 26 13:24 debug_trainTextMining_lenRel.csv.gz
-rw-r--r-- 1 kesci users  21M Jun 24 12:15 debug_trainTextMining_ngramCharSim.csv.gz
-rw-r--r-- 1 kesci users  34M Jun 25 14:00 debug_trainTextMining_ngramSim.csv.gz
-rw-r--r-- 1 kesci users  20M Jun 26 13:30 debug_trainTextMining_ngramSimV1.csv.gz
-rw-r--r-- 1 kesci users 2.4M Jun 29 13:24 debug_trainTextMining_ngramSubSeq.csv.gz
-rw-r--r-- 1 kesci users 6.6M Jun 29 13:47 debug_trainTextMining_ngramSubSeqV1.csv.gz
-rw-r--r-- 1 kesci users  86M Jun 25 17:42 online_testTextMining_editSim.csv.gz
-rw-r--r-- 1 kesci users  47M Jun 20 13:22 online_testTextMining_len.csv.gz
-rw-r--r-- 1 kesci users  27M Jun 25 16:09 online_testTextMining_lenRel.csv.gz
-rw-r--r-

### 提取长度特征

In [11]:
regen_len_feature = False

In [12]:
def run_text_len(df, ngram, prefix):
    # ngram
    if ngram:
        q_list = df['query'].apply(ngram)
        t_list = df['title'].apply(ngram)
    else:
        q_list = df['query']
        t_list = df['title']
    ## 长度特征
    with Timer("extract length"):
        df['%s_q-len' % prefix] = np.vectorize(len)(q_list)
        df['%s_t-len' % prefix] = np.vectorize(len)(t_list)
        df['%s_qt-len-radio'%prefix] = list(map(lambda a, b: a/b, 
            df['%s_q-len' % prefix], 
            df['%s_t-len' % prefix]))
        df['%s_tq-len-radio'%prefix] = list(map(lambda a, b: b/a, 
            df['%s_q-len' % prefix], 
            df['%s_t-len' % prefix]))
    del q_list, t_list
    gc.collect()
    return df
def process_text_len(df, prefix, savefile):
    df = run_text_len(df, None, '%s_%s' % (prefix, 'text'))
    df = run_text_len(df, ngram_utils.unigrams, '%s_%s' % (prefix, 'unigrams'))
    return df

In [13]:
# 提取训练数据特征
if regen_len_feature:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "len", process_func=process_text_len, 
        names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], drop_last_cols=['query', 'title'])

In [7]:
# AnalysisCSV("./stage1/output/text_mining_feature/debug_trainTextMining_len.csv.gz")

In [6]:
if not OFFLINE and regen_len_feature:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, "len", process_func=process_text_len, 
    names=ORI_TEST_NAMES, dtype=ORI_TEST_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
    drop_first_cols=['query_id', 'query_title_id'], drop_last_cols=['query', 'title'])

In [11]:
# AnalysisCSV("./stage1/output/text_mining_feature/debug_train_text_mining_feature_len.csv")

time: 989 µs


### 提取更多长度特征

In [7]:
regen_len_rel_feature = False

In [8]:
def run_text_len_rel(df, ngram, prefix):
    # ngram
    q_list = df['query'].apply(ngram)
    t_list = df['title'].apply(ngram)
    ## 长度特征
    with Timer("extract length rel"):
        q_len = np.vectorize(len)(q_list)
        t_len = np.vectorize(len)(t_list)
        del q_list, t_list
        gc.collect()
        df['%s_qt-diff'%prefix] = list(map(lambda a, b: abs(a-b), q_len, t_len))
        df['%s_qt-max'%prefix] = list(map(lambda a, b: max(a, b), q_len, t_len))
        df['%s_qt-min'%prefix] = list(map(lambda a, b: min(a, b), q_len, t_len))
        df['%s_qt-avg'%prefix] = list(map(lambda a, b: (a+b)/2, q_len, t_len))
        del q_len, t_len
        gc.collect()
    return df
def process_text_len_rel(df, prefix, savefile):
    df = run_text_len_rel(df, ngram_utils.unichars, '%s_%s' % (prefix, 'unichars'))
    df = run_text_len_rel(df, ngram_utils.unigrams, '%s_%s' % (prefix, 'unigrams'))
    return df

In [9]:
# 提取训练数据特征
if regen_len_rel_feature:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "lenRel", 
        process_func=process_text_len_rel, 
        names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], drop_last_cols=['query', 'title'])

----->Started 'extract lenRel feature' block...
Mem. usage decreased to  7.63 Mb (0.0% reduction)
----->Started 'extract length rel' block...
----->Finished 'extract length rel' block, time used:3.31s.
----->Started 'extract length rel' block...
----->Finished 'extract length rel' block, time used:3.9s.
saved lenRel feature to ./stage1/output/text_mining_feature/debug_trainTextMining_lenRel.csv.gz
----->Finished 'extract lenRel feature' block, time used:27.86s.


In [15]:
# AnalysisCSV("./stage1/output/text_mining_feature/debug_trainTextMining_lenRel.csv.gz")

In [13]:
if not OFFLINE and regen_len_rel_feature:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, "lenRel", 
    process_func=process_text_len_rel, 
    names=ORI_TEST_NAMES, dtype=ORI_TEST_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
    drop_first_cols=['query_id', 'query_title_id'], drop_last_cols=['query', 'title'])

----->Started 'extract lenRel feature' block...
Mem. usage decreased to 76.29 Mb (0.0% reduction)
----->Started 'extract length rel' block...
----->Finished 'extract length rel' block, time used:19.74s.
----->Started 'extract length rel' block...
----->Finished 'extract length rel' block, time used:19.97s.
saved lenRel feature to ./stage1/output/text_mining_feature/online_testTextMining_lenRel.csv.gz
----->Finished 'extract lenRel feature' block, time used:180.14s.


### 提取编辑距离特征

In [10]:
regen_edit_sim_feature = False
import Levenshtein

In [11]:
similar_arr = [
    # 编辑距离，https://www.jb51.net/article/98449.htm;
    # http://www.coli.uni-saarland.de/courses/LT1/2011/slides/Python-Levenshtein.html#Levenshtein-inverse
    Levenshtein.distance,
    Levenshtein.jaro,
    Levenshtein.jaro_winkler,
    # Levenshtein.ratio
    # https://blog.csdn.net/qq_43174128/article/details/82595317
    fuzz.ratio,
    fuzz.partial_ratio,
    fuzz.token_sort_ratio,
    fuzz.partial_token_sort_ratio,
    fuzz.token_set_ratio
]

In [12]:
def run_edit_sim_feature(df, prefix):
    q_list = df['query']
    t_list = df['title']
    ## 长度特征
    with Timer("extract edit sim"):
        for _ in similar_arr:
            name = _.__name__
            with Timer("cal {} sim".format(name)):
                df["{}_{}".format(prefix, name)] = cal_sim(q_list, t_list, _)
    del q_list, t_list
    gc.collect()
    return df

def process_edit_sim_feature(df, prefix, savefile):
    df = run_edit_sim_feature(df, '%s_%s' % (prefix, 'text'))
    return df

In [13]:
# 提取训练数据特征
if regen_edit_sim_feature:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "editSim", 
        process_func=process_edit_sim_feature, 
        names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], drop_last_cols=['query', 'title'])

----->Started 'extract editSim feature' block...
Mem. usage decreased to  7.63 Mb (0.0% reduction)
----->Started 'extract edit sim' block...
----->Started 'cal distance sim' block...
----->Finished 'cal distance sim' block, time used:2.55s.
----->Started 'cal jaro sim' block...
----->Finished 'cal jaro sim' block, time used:1.1s.
----->Started 'cal jaro_winkler sim' block...
----->Finished 'cal jaro_winkler sim' block, time used:1.26s.
----->Started 'cal ratio sim' block...
----->Finished 'cal ratio sim' block, time used:5.63s.
----->Started 'cal partial_ratio sim' block...
----->Finished 'cal partial_ratio sim' block, time used:27.78s.
----->Started 'cal token_sort_ratio sim' block...
----->Finished 'cal token_sort_ratio sim' block, time used:22.18s.
----->Started 'cal partial_token_sort_ratio sim' block...
----->Finished 'cal partial_token_sort_ratio sim' block, time used:49.13s.
----->Started 'cal token_set_ratio sim' block...
----->Finished 'cal token_set_ratio sim' block, time use

In [34]:
if not OFFLINE and regen_edit_sim_feature:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, "editSim", 
    process_func=process_edit_sim_feature, 
    names=ORI_TEST_NAMES, dtype=ORI_TEST_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
    drop_first_cols=['query_id', 'query_title_id'], drop_last_cols=['query', 'title'])

----->Started 'extract editSim feature' block...
Mem. usage decreased to 76.29 Mb (0.0% reduction)
----->Started 'extract edit sim' block...
----->Started 'cal distance sim' block...
----->Finished 'cal distance sim' block, time used:18.15s.
----->Started 'cal jaro sim' block...
----->Finished 'cal jaro sim' block, time used:6.74s.
----->Started 'cal jaro_winkler sim' block...
----->Finished 'cal jaro_winkler sim' block, time used:7.09s.
----->Started 'cal ratio sim' block...
----->Finished 'cal ratio sim' block, time used:34.9s.
----->Started 'cal partial_ratio sim' block...
----->Finished 'cal partial_ratio sim' block, time used:175.09s.
----->Started 'cal token_sort_ratio sim' block...
----->Finished 'cal token_sort_ratio sim' block, time used:140.28s.
----->Started 'cal partial_token_sort_ratio sim' block...
----->Finished 'cal partial_token_sort_ratio sim' block, time used:288.69s.
----->Started 'cal token_set_ratio sim' block...
----->Finished 'cal token_set_ratio sim' block, tim

### 提取集合相似度特征

In [14]:
regen_ngram_sim_feature = False
import Levenshtein

In [15]:
similar_arr = [
    # 集合的交集关系
    # 可以操作字符串，如"abc"；也可以操作字符数组，如['a', 'b', 'c']
    dist_utils.dice_ratio,
    dist_utils.jaccard_ratio,
    # 编辑距离，https://www.jb51.net/article/98449.htm;
    # http://www.coli.uni-saarland.de/courses/LT1/2011/slides/Python-Levenshtein.html#Levenshtein-inverse
    dist_utils.edit_seq_ratio,
    dist_utils.edit_set_ratio,
]

In [16]:
def run_ngram_similarity(df, ngram, prefix):
    q_list = df['query'].apply(ngram)
    t_list = df['title'].apply(ngram)
    ## 长度特征
    with Timer("extract ngram sim"):
        for _ in similar_arr:
            name = _.__name__
            with Timer("cal {} sim".format(name)):
                df["{}_{}".format(prefix, name)] = cal_sim(q_list, t_list, _)
        del q_list, t_list
        gc.collect()
    return df

def process_ngram_similarity(df, prefix, savefile):
    df = run_ngram_similarity(df, ngram_utils.unichars, '%s_%s' % (prefix, 'unichars'))
    # df = run_ngram_similarity(df, ngram_utils.bichars, '%s_%s' % (prefix, 'bichars'))
    # df = run_ngram_similarity(df, ngram_utils.trichars, '%s_%s' % (prefix, 'trichars'))
    df = run_ngram_similarity(df, ngram_utils.unigrams, '%s_%s' % (prefix, 'unigrams'))
    df = run_ngram_similarity(df, ngram_utils.bigrams, '%s_%s' % (prefix, 'bigrams'))
    df = run_ngram_similarity(df, ngram_utils.trigrams, '%s_%s' % (prefix, 'trigrams'))
    return df

In [17]:
# 提取训练数据特征
if regen_ngram_sim_feature:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "ngramSimV1", 
        process_func=process_ngram_similarity, 
        names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], drop_last_cols=['query', 'title'])

----->Started 'extract ngramSimV1 feature' block...
Mem. usage decreased to  7.63 Mb (0.0% reduction)
----->Started 'extract ngram sim' block...
----->Started 'cal dice_ratio sim' block...
----->Finished 'cal dice_ratio sim' block, time used:2.3s.
----->Started 'cal jaccard_ratio sim' block...
----->Finished 'cal jaccard_ratio sim' block, time used:2.46s.
----->Started 'cal edit_seq_ratio sim' block...
----->Finished 'cal edit_seq_ratio sim' block, time used:9.89s.
----->Started 'cal edit_set_ratio sim' block...
----->Finished 'cal edit_set_ratio sim' block, time used:13.76s.
----->Finished 'extract ngram sim' block, time used:28.78s.
----->Started 'extract ngram sim' block...
----->Started 'cal dice_ratio sim' block...
----->Finished 'cal dice_ratio sim' block, time used:2.29s.
----->Started 'cal jaccard_ratio sim' block...
----->Finished 'cal jaccard_ratio sim' block, time used:2.0s.
----->Started 'cal edit_seq_ratio sim' block...
----->Finished 'cal edit_seq_ratio sim' block, time u

In [41]:
# AnalysisCSV("./stage1/output/text_mining_feature/debug_trainTextMining_ngramSim.csv.gz")

In [7]:
if not OFFLINE and regen_ngram_sim_feature:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, "ngramSimV1", 
    process_func=process_ngram_similarity, 
    names=ORI_TEST_NAMES, dtype=ORI_TEST_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
    drop_first_cols=['query_id', 'query_title_id'], drop_last_cols=['query', 'title'])

----->Started 'extract ngramSimChars feature' block...
Mem. usage decreased to 76.29 Mb (0.0% reduction)
----->Started 'extract ngram sim' block...
----->Started 'cal dice_ratio sim' block...
----->Finished 'cal dice_ratio sim' block, time used:15.78s.
----->Started 'cal jaccard_ratio sim' block...
----->Finished 'cal jaccard_ratio sim' block, time used:15.62s.
----->Started 'cal edit_seq_ratio sim' block...
----->Finished 'cal edit_seq_ratio sim' block, time used:59.22s.
----->Started 'cal edit_set_ratio sim' block...
----->Finished 'cal edit_set_ratio sim' block, time used:85.41s.
----->Finished 'extract ngram sim' block, time used:178.06s.
----->Started 'extract ngram sim' block...
----->Started 'cal dice_ratio sim' block...
----->Finished 'cal dice_ratio sim' block, time used:15.53s.
----->Started 'cal jaccard_ratio sim' block...
----->Finished 'cal jaccard_ratio sim' block, time used:12.36s.
----->Started 'cal edit_seq_ratio sim' block...
----->Finished 'cal edit_seq_ratio sim' bl

### SimHash

In [12]:
from simhash import Simhash
# https://github.com/cjauvin/simhash/blob/master/tests/test_simhash.py
# https://leons.im/posts/a-python-implementation-of-simhash-algorithm/
regen_simhash = True
tfidf_model_file = "./stage1/output/vector_space/debug_tfidf_model.bin"

In [13]:
def process_ngram_simhash(df, prefix, savefile):
    with Timer("cal tfidf vec"):
        with open(tfidf_model_file, "rb") as ff:
            tfidf_model = pickle.load(ff)
        q_vecs = tfidf_model.transform(df['query'])
        t_vecs = tfidf_model.transform(df['title'])
        voc = dict((i, w) for w, i in tfidf_model.vocabulary_.items())
    with Timer("cal sim hash"):
        q_simhash = [
            Simhash(
                zip([voc[j] for j in Di.indices], Di.data)
            )
            for Di in q_vecs
        ]
        t_simhash = [
            Simhash(
                zip([voc[j] for j in Di.indices], Di.data)
            )
            for Di in t_vecs
        ]
    with Timer("cal sim hash sim"):
        df["{}Simhash".format(prefix)] = list(map(
            lambda x,y: x.distance(y), 
            q_simhash, t_simhash))
        
    del q_vecs, t_vecs, q_simhash, t_simhash
    gc.collect()
    return df

# def process_ngram_simhash(df, prefix, savefile):
#     # df = run_ngram_simhash(df, ngram_utils.unichars, '%s_%s' % (prefix, 'unichars'))
#     # df = run_ngram_simhash(df, ngram_utils.bichars, '%s_%s' % (prefix, 'bichars'))
#     # df = run_ngram_simhash(df, ngram_utils.trichars, '%s_%s' % (prefix, 'trichars'))
#     df = run_ngram_simhash(df, ngram_utils.unigrams, '%s_%s' % (prefix, 'unigrams'))
#     df = run_ngram_simhash(df, ngram_utils.bigrams, '%s_%s' % (prefix, 'bigrams'))
#     # df = run_ngram_simhash(df, ngram_utils.trigrams, '%s_%s' % (prefix, 'trigrams'))
#     return df

In [14]:
# 提取训练数据特征
if regen_simhash:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "ngramSimHashV1", 
        process_func=process_ngram_simhash, 
        names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], drop_last_cols=['query', 'title'])

----->Started 'extract ngramSimHashV1 feature' block...
Mem. usage decreased to  7.63 Mb (0.0% reduction)
----->Started 'cal tfidf vec' block...
----->Finished 'cal tfidf vec' block, time used:27.81s.
----->Started 'cal sim hash' block...
----->Finished 'cal sim hash' block, time used:517.59s.
saved ngramSimHashV1 feature to ./stage1/output/vector_space/debug_trainVectorSpace_ngramSimHashV1.csv.gz
----->Finished 'extract ngramSimHashV1 feature' block, time used:549.25s.


In [16]:
AnalysisCSV("./stage1/output/vector_space/debug_trainVectorSpace_ngramSimHashV1.csv.gz", print_rows=10)

--->file shape:
(500000, 1)
--->10 rows of file:
   debug_trainVectorSpaceSimhash
0                             24
1                             25
2                             20
3                             28
4                             28
5                             29
6                             31
7                             24
8                             21
9                             31
--->file info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 1 columns):
debug_trainVectorSpaceSimhash    500000 non-null int64
dtypes: int64(1)
memory usage: 3.8 MB
None
--->file describe:
       debug_trainVectorSpaceSimhash
count                  500000.000000
mean                       26.757976
std                         5.525289
min                         0.000000
25%                        23.000000
50%                        27.000000
75%                        31.000000
max                        49.000000


### 最长公共子串

In [4]:
import py_common_subseq
regen_subseq = True

In [5]:
def run_ngram_subseq(df, ngram, prefix):
    with Timer("cal sub seq"):
        query = df["query"].apply(ngram)
        title = df["title"].apply(ngram)
        df["{}SubSeqCount".format(prefix)] = np.log(np.array(list(map(
            lambda x, y: py_common_subseq.count_common_subsequences(x, y), 
            query, title))).astype(float))
        del query, title
        gc.collect()
    return df

def process_ngram_subseq(df, prefix, savefile):
    df = run_ngram_subseq(df, ngram_utils.unichars, '%s_%s' % (prefix, 'unichars'))
    df = run_ngram_subseq(df, ngram_utils.bichars, '%s_%s' % (prefix, 'bichars'))
    df = run_ngram_subseq(df, ngram_utils.trichars, '%s_%s' % (prefix, 'trichars'))
    # df = run_ngram_subseq(df, ngram_utils.unigrams, '%s_%s' % (prefix, 'unigrams'))
    # df = run_ngram_subseq(df, ngram_utils.bigrams, '%s_%s' % (prefix, 'bigrams'))
    # df = run_ngram_subseq(df, ngram_utils.trigrams, '%s_%s' % (prefix, 'trigrams'))
    return df 

In [15]:
# 提取训练数据特征
if regen_subseq:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "ngramSubSeq", 
        process_func=process_ngram_subseq, 
        names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], drop_last_cols=['query', 'title'])

----->Started 'extract ngramSubSeq feature' block...
Mem. usage decreased to 152.59 Mb (0.0% reduction)
----->Started 'cal sub seq' block...


In [16]:
# AnalysisCSV("./stage1/output/text_mining_feature/debug_trainTextMining_ngramSubSeqV1.csv.gz")

In [ ]:
# 提取训练数据特征
if regen_subseq:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, "ngramSubSeq", 
        process_func=process_ngram_subseq, 
        names=ORI_TEST_NAMES, dtype=ORI_TEST_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], drop_last_cols=['query', 'title'])

### textRank

In [ ]:
from gensim.summarization.summarizer import summarize
regen_textrank = True

## 向量空间特征

In [2]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
feature_save_dir = base_feature_save_dir + "vector_space/"
train_feature_prefix = base_prefix + "trainVectorSpace"
test_feature_prefix = base_prefix + "testVectorSpace"
tfidf_model_file = feature_save_dir+"online_tfidf_model_ngram4.bin"
print(tfidf_model_file)
! ls -alh ./stage1/output/vector_space/

./stage1/output/vector_space/online_tfidf_model_ngram4.bin
total 6.8G
drwxr-xr-x 2 kesci root  4.0K Jun 30 02:58 .
drwxr-xr-x 7 kesci root  4.0K Jun 19 09:08 ..
-rw-r--r-- 1 kesci users  89M Jun 20 11:43 debug_tfidf_model.bin
-rw-r--r-- 1 kesci users 1.2M Jun 28 14:44 debug_trainVectorSpace_ngramSimHash.csv.gz
-rw-r--r-- 1 kesci users 374K Jun 29 02:52 debug_trainVectorSpace_ngramSimHashV1.csv.gz
-rw-r--r-- 1 kesci users  30M Jun 29 07:22 debug_trainVectorSpace_tfidfVecLenChar.csv.gz
-rw-r--r-- 1 kesci users  25M Jun 27 14:25 debug_trainVectorSpace_tfidfVecLen.csv.gz
-rw-r--r-- 1 kesci users  31M Jun 30 02:58 debug_trainVectorSpace_tfidfVecLenMean.csv.gz
-rw-r--r-- 1 kesci users  18M Jun 28 12:50 debug_trainVectorSpace_tfidfVecLenMore.csv.gz
-rw-r--r-- 1 kesci users 4.1M Jun 28 12:59 debug_trainVectorSpace_tfidfVecLenMulti.csv.gz
-rw-r--r-- 1 kesci users  30M Jun 29 06:37 debug_trainVectorSpace_tfidfVecLenV1.csv.gz
-rw-r--r-- 1 kesci users  31M Jun 30 02:16 debug_trainVectorSpace_tfidf

In [8]:
! rm ./stage1/output/vector_space/online_tfidf_model.bin

### 训练tfidf模型

In [3]:
regen_tfidf_model = True
print(train_file)

./stage1/input/train_last_500w.csv


In [4]:
def train_tfidf(df, prefix, savefile):
    word_tfidf = TfidfVectorizer(norm="l2",
                                    strip_accents="unicode",
                                    analyzer="word",
                                    ngram_range=(1, 4),
                                    use_idf=True,
                                    smooth_idf=True,
                                    sublinear_tf=True, min_df=5, max_df=0.9)
    new_query = df["query"].unique()
    new_title = df["title"].values
    # print(new_query.shape)
    # print(new_title.shape)
    corpus = np.concatenate([new_query, new_title])
    # print(corpus[:10])
    # print(corpus[10:])
    del df
    gc.collect()
    with Timer("tf-idf fit"):
        word_tfidf.fit(corpus)
        with open(tfidf_model_file, "wb") as f:
            pickle.dump(word_tfidf, f)
            print("Model dumped to {}".format(tfidf_model_file))
    return None

In [5]:
# 提取训练数据特征
if regen_tfidf_model:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "tfidf", 
        process_func=train_tfidf, names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, 
        process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], 
        drop_last_cols=['query', 'title'])

----->Started 'extract tfidf feature' block...
Mem. usage decreased to 76.29 Mb (0.0% reduction)
----->Started 'tf-idf fit' block...


### tfidf向量相似性

In [14]:
regen_tfidf_vec_sim = True
print(tfidf_model_file)

./stage1/output/vector_space/online_tfidf_model_ngram3.bin


In [15]:
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise
from sklearn.metrics.pairwise import *
similar_dis = [
    paired_cosine_distances,
    paired_euclidean_distances,
    paired_manhattan_distances
]

In [16]:
def process_tfidf_vec_sim(df, prefix, savefile):
    with Timer("cal tfidf vec"):
        with open(tfidf_model_file, "rb") as ff:
            tfidf_model = pickle.load(ff)
        q_vec = tfidf_model.transform(df["query"])
        t_vec = tfidf_model.transform(df["title"])
    with Timer("cal tfidf sim"):
        for _ in similar_dis:
            name = _.__name__
            with Timer("cal {} sim".format(name)):
                df["{}_{}".format(prefix, name)] = _(q_vec, t_vec)
    return df

In [17]:
# 提取训练数据特征
if regen_tfidf_vec_sim:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "tfidfVecSimV1", 
        process_func=process_tfidf_vec_sim, names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, 
        process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], 
        drop_last_cols=['query', 'title'])

----->Started 'extract tfidfVecSimV1 feature' block...
Mem. usage decreased to 152.59 Mb (0.0% reduction)
----->Started 'cal tfidf vec' block...
----->Finished 'cal tfidf vec' block, time used:956.27s.
----->Started 'cal tfidf sim' block...
----->Started 'cal paired_cosine_distances sim' block...
----->Finished 'cal paired_cosine_distances sim' block, time used:25.42s.
----->Started 'cal paired_euclidean_distances sim' block...
----->Finished 'cal paired_euclidean_distances sim' block, time used:20.32s.
----->Started 'cal paired_manhattan_distances sim' block...
----->Finished 'cal paired_manhattan_distances sim' block, time used:22.01s.
----->Finished 'cal tfidf sim' block, time used:67.75s.
saved tfidfVecSimV1 feature to ./stage1/output/vector_space/online_trainVectorSpace_tfidfVecSimV1.csv.gz
----->Finished 'extract tfidfVecSimV1 feature' block, time used:1230.26s.


In [18]:
# AnalysisCSV("./stage1/output/vector_space/debug_trainVectorSpace_tfidfVecSimV1.csv.gz")

In [19]:
# 提取训练数据特征
if not OFFLINE and regen_tfidf_vec_sim:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, "tfidfVecSimV1", 
        process_func=process_tfidf_vec_sim, names=ORI_TEST_NAMES, dtype=ORI_TEST_DTYPE, 
        process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], 
        drop_last_cols=['query', 'title'])

----->Started 'extract tfidfVecSimV1 feature' block...
Mem. usage decreased to 76.29 Mb (0.0% reduction)
----->Started 'cal tfidf vec' block...
----->Finished 'cal tfidf vec' block, time used:515.43s.
----->Started 'cal tfidf sim' block...
----->Started 'cal paired_cosine_distances sim' block...
----->Finished 'cal paired_cosine_distances sim' block, time used:12.92s.
----->Started 'cal paired_euclidean_distances sim' block...
----->Finished 'cal paired_euclidean_distances sim' block, time used:10.5s.
----->Started 'cal paired_manhattan_distances sim' block...
----->Finished 'cal paired_manhattan_distances sim' block, time used:11.16s.
----->Finished 'cal tfidf sim' block, time used:34.58s.
saved tfidfVecSimV1 feature to ./stage1/output/vector_space/online_testVectorSpace_tfidfVecSimV1.csv.gz
----->Finished 'extract tfidfVecSimV1 feature' block, time used:666.02s.


### tfidf长度特征

In [3]:
from np_utils import try_divide
regen_tfidf_vec_len = True
import scipy, numpy
print(tfidf_model_file)

./stage1/output/vector_space/online_tfidf_model_ngram4.bin


In [5]:
def process_tfidf_vec_len(df, prefix, savefile):
    with Timer("cal tfidf vec"):
        with open(tfidf_model_file, "rb") as ff:
            tfidf_model = pickle.load(ff)
        q_vec_len = np.array(tfidf_model.transform(df["query"]).sum(1)).squeeze()
        t_vec_len = np.array(tfidf_model.transform(df["title"]).sum(1)).squeeze()
    with Timer("cal tfidf len"):
        # Len
        df['%s_QLen' % prefix] = q_vec_len
        df['%s_TLen' % prefix] = t_vec_len
        df['%s_QTLenRatio'%prefix] = list(map(lambda a, b: try_divide(a, b), q_vec_len, t_vec_len))
        df['%s_TQLenRatio'%prefix] = list(map(lambda a, b: try_divide(b, a), q_vec_len, t_vec_len))
        df['%s_QTDiff'%prefix] = list(map(lambda a, b: abs(a-b), q_vec_len, t_vec_len))
        df['%s_QTMax'%prefix] = list(map(lambda a, b: max(a, b), q_vec_len, t_vec_len))
        df['%s_QTMin'%prefix] = list(map(lambda a, b: min(a, b), q_vec_len, t_vec_len))
        df['%s_QTAvg'%prefix] = list(map(lambda a, b: (a+b)/2, q_vec_len, t_vec_len))
        df['%s_QTMulti'%prefix] = list(map(lambda a, b: a*b, q_vec_len, t_vec_len))
        # df['%s_QSquare2'%prefix] = numpy.square(q_vec_len)
        # df['%s_TSquare2'%prefix] = numpy.square(t_vec_len)
        # df['%s_QSqrt'%prefix] = numpy.sqrt(q_vec_len)
        # df['%s_TSqrt'%prefix] = numpy.sqrt(t_vec_len)
        # df['%s_QLog'%prefix] = numpy.log(q_vec_len)
        # df['%s_TLog'%prefix] = numpy.log(t_vec_len)
        # p_corr = scipy.stats.pearsonr(q_vec_len, t_vec_len)[0]
        # df['%s_PCorr'%prefix] = [p_corr] * len(q_vec_len)
        del q_vec_len, t_vec_len
        gc.collect()
    return df

In [5]:
# 提取训练数据特征
if regen_tfidf_vec_len:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "tfidfVecLenMean", 
        process_func=process_tfidf_vec_len, names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, 
        process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], 
        drop_last_cols=['query', 'title'])

----->Started 'extract tfidfVecLenMean feature' block...
Mem. usage decreased to  7.63 Mb (0.0% reduction)
----->Started 'cal tfidf vec' block...
----->Finished 'cal tfidf vec' block, time used:87.16s.
----->Started 'cal tfidf len' block...
----->Finished 'cal tfidf len' block, time used:9.81s.
saved tfidfVecLenMean feature to ./stage1/output/vector_space/debug_trainVectorSpace_tfidfVecLenMean.csv.gz
----->Finished 'extract tfidfVecLenMean feature' block, time used:142.81s.


In [3]:
# AnalysisCSV("./stage1/output/vector_space/debug_trainVectorSpace_tfidfVecLenV1.csv.gz")

In [6]:
# 提取训练数据特征
if not OFFLINE and regen_tfidf_vec_len:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, "tfidfVecLenV2", 
        process_func=process_tfidf_vec_len, names=ORI_TEST_NAMES, dtype=ORI_TEST_DTYPE, 
        process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id'], 
        drop_last_cols=['query', 'title'])

----->Started 'extract tfidfVecLenV2 feature' block...
Mem. usage decreased to 76.29 Mb (0.0% reduction)
----->Started 'cal tfidf vec' block...
----->Finished 'cal tfidf vec' block, time used:540.42s.
----->Started 'cal tfidf len' block...
----->Finished 'cal tfidf len' block, time used:36.82s.
saved tfidfVecLenV2 feature to ./stage1/output/vector_space/online_testVectorSpace_tfidfVecLenV2.csv.gz
----->Finished 'extract tfidfVecLenV2 feature' block, time used:760.42s.


## 提取word2vec特征

In [9]:
import pickle
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models.callbacks import CallbackAny2Vec
from scipy.stats import skew, kurtosis
word2vec_model_file = "./stage1/input/word2vec.kv"
tfidf_model_file = "./stage1/output/vector_space/online_tfidf_model.bin"
feature_save_dir = base_feature_save_dir + "word2vec/"
train_feature_prefix = base_prefix + "trainWord2vec"
test_feature_prefix = base_prefix + "testWord2vec"
! ls -alh ./stage1/output/word2vec

total 2.0G
drwxr-xr-x 2 kesci root  4.0K Jun 28 12:16 .
drwxr-xr-x 7 kesci root  4.0K Jun 19 09:08 ..
-rw-r--r-- 1 kesci users  23M Jun 25 14:33 debug_trainWord2vec_senVecavgSim.csv.gz
-rw-r--r-- 1 kesci users  26M Jun 26 13:39 debug_trainWord2vec_senVecavgSimV1.csv.gz
-rw-r--r-- 1 kesci users  12M Jun 23 14:53 debug_trainWord2vec_senVecavgSK.csv.gz
-rw-r--r-- 1 kesci users 8.8M Jun 23 17:46 debug_trainWord2vec_senVecavgWmd.csv.gz
-rw-r--r-- 1 kesci users  23M Jun 25 14:55 debug_trainWord2vec_senVectfidfSim.csv.gz
-rw-r--r-- 1 kesci users  25M Jun 28 12:05 debug_trainWord2vec_senVectfidfSimV1.csv.gz
-rw-r--r-- 1 kesci users  25M Jun 28 12:16 debug_trainWord2vec_senVectfidfSimV1Sum.csv.gz
-rw-r--r-- 1 kesci users 282M Jun 21 12:33 online_testWord2vec_senVecavgSim.csv.gz
-rw-r--r-- 1 kesci users 251M Jun 27 03:59 online_testWord2vec_senVecavgSimV1.csv.gz
-rw-r--r-- 1 kesci users 564M Jun 21 09:21 online_trainWord2vec_senVecavgSim.csv.gz
-rw-r--r-- 1 kesci users 502M Jun 27 03:05 online_t

In [25]:
def sent2vec1(text):
    # 计算embedding均值
    global word2vec_model, tfidf_idf, tfidf_vocab
    words = text.split()
    words_num = len(words)
    return np.nan_to_num(
          np.array([np.array(word2vec_model[w])*(
             (1/words_num) * tfidf_idf[tfidf_vocab[w]]
             ) for w in words 
             if w in word2vec_model and w in tfidf_vocab] or [0.0] * 200
          ).sum(axis=0)
      )

In [5]:
def sent2vec(text):
    # 计算embedding均值
    global word2vec_model
    return np.nan_to_num(
          np.array([word2vec_model[w] for w in text.split() if w in word2vec_model] or [0.0] * 200
          ).mean(axis=0)
      )

### 生成word2vec句子相似度

In [21]:
regen_word2vec_sen_vec_sim = True
process_word2vec_sen_vec_mode = "tfidf"
# word2vec to sent: https://www.zhihu.com/question/29978268

In [22]:
similar_dis = [
    dist_utils.cosine_distance, 
    dist_utils.jaccard_distance,
    dist_utils.braycurtis_distance,
    dist_utils.canberra_distance,
    dist_utils.cityblock_distance,
    dist_utils.euclidean_distance,
    dist_utils.minkowski_distance
]
similar_arr = similar_dis

In [16]:
if regen_word2vec_sen_vec_sim:
    word2vec_model = KeyedVectors.load(word2vec_model_file, mmap='r')

In [17]:
if regen_word2vec_sen_vec_sim and process_word2vec_sen_vec_mode == "tfidf":
    with open(tfidf_model_file, "rb") as ff:
        tfidf_model = pickle.load(ff)
    tfidf_idf = tfidf_model.idf_
    tfidf_vocab = tfidf_model.vocabulary_

In [19]:
print(tfidf_vocab["13"])
print(tfidf_idf[:10])
print(tfidf_idf[tfidf_vocab["13"]])

993383
[ 5.15864267 13.26993733  8.96762552 14.42570803 16.16030909 12.98225526
 14.42570803 15.87262701 15.31301123 15.87262701]
4.024379974189072


In [23]:
def process_word2vec_sen_vec(df, prefix, savefile):
    with Timer("cal sent vec"):
        global process_word2vec_sen_vec_mode
        if process_word2vec_sen_vec_mode == "avg":
            q_sen_vec = list(map(sent2vec, df["query"]))
            t_sen_vec = list(map(sent2vec, df["title"]))
        elif process_word2vec_sen_vec_mode == "tfidf":
            q_sen_vec = list(map(sent2vec1, df["query"]))
            t_sen_vec = list(map(sent2vec1, df["title"]))
        # savefile += "{}.npz".format(process_word2vec_sen_vec_mode)
        # np.savez(savefile, q_sent_vec, t_sent_vec)
        # print("Sent vec saved to {}".format(savefile))
        for _ in similar_arr:
            name = _.__name__
            with Timer("cal {} sim".format(name)):
                df["{}_{}".format(prefix, name)] = cal_sim(q_sen_vec, t_sen_vec, _)
        del q_sen_vec, t_sen_vec
        gc.collect()
    return df

In [26]:
# 提取训练数据特征
if regen_word2vec_sen_vec_sim:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, 
        "senVec{}SimV1Sum".format(process_word2vec_sen_vec_mode), 
        process_func=process_word2vec_sen_vec, names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, 
        process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], 
        drop_last_cols=['query', 'title'])

----->Started 'extract senVectfidfSimV1Sum feature' block...
Mem. usage decreased to  7.63 Mb (0.0% reduction)
----->Started 'cal sent vec' block...
----->Started 'cal cosine_distance sim' block...
----->Finished 'cal cosine_distance sim' block, time used:33.73s.
----->Started 'cal jaccard_distance sim' block...
----->Finished 'cal jaccard_distance sim' block, time used:27.93s.
----->Started 'cal braycurtis_distance sim' block...
----->Finished 'cal braycurtis_distance sim' block, time used:28.27s.
----->Started 'cal canberra_distance sim' block...
----->Finished 'cal canberra_distance sim' block, time used:41.03s.
----->Started 'cal cityblock_distance sim' block...
----->Finished 'cal cityblock_distance sim' block, time used:21.92s.
----->Started 'cal euclidean_distance sim' block...
----->Finished 'cal euclidean_distance sim' block, time used:26.94s.
----->Started 'cal minkowski_distance sim' block...
----->Finished 'cal minkowski_distance sim' block, time used:43.05s.
----->Finished

In [12]:
# 提取训练数据特征
if regen_word2vec_sen_vec_sim and not OFFLINE:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, 
        "senVec{}SimV1".format(process_word2vec_sen_vec_mode), 
        process_func=process_word2vec_sen_vec, names=ORI_TEST_NAMES, dtype=ORI_TEST_DTYPE, 
        process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id'], 
        drop_last_cols=['query', 'title'])

----->Started 'extract senVecavgSimV1 feature' block...
Mem. usage decreased to 76.29 Mb (0.0% reduction)
----->Started 'cal sent vec' block...
----->Started 'cal cosine_distance sim' block...
----->Finished 'cal cosine_distance sim' block, time used:239.94s.
----->Started 'cal jaccard_distance sim' block...
----->Finished 'cal jaccard_distance sim' block, time used:257.73s.
----->Started 'cal braycurtis_distance sim' block...


/opt/conda/lib/python3.6/site-packages/scipy/spatial/distance.py:1160: RuntimeWarning: invalid value encountered in double_scalars
  return l1_diff.sum() / l1_sum.sum()


----->Finished 'cal braycurtis_distance sim' block, time used:251.68s.
----->Started 'cal canberra_distance sim' block...
----->Finished 'cal canberra_distance sim' block, time used:383.57s.
----->Started 'cal cityblock_distance sim' block...
----->Finished 'cal cityblock_distance sim' block, time used:189.13s.
----->Started 'cal euclidean_distance sim' block...
----->Finished 'cal euclidean_distance sim' block, time used:248.57s.
----->Started 'cal minkowski_distance sim' block...
----->Finished 'cal minkowski_distance sim' block, time used:420.52s.
----->Finished 'cal sent vec' block, time used:3090.11s.
saved senVecavgSimV1 feature to ./stage1/output/word2vec/online_testWord2vec_senVecavgSimV1.csv.gz
----->Finished 'extract senVecavgSimV1 feature' block, time used:3244.02s.


In [74]:
# AnalysisCSV("./stage1/output/word2vec/debug_trainWord2vec_senVecavgSim.csv.gz")

### word2vec其他特征

In [10]:
word2vec_model_norm = None
regen_word2vec_sen_vec_other = False
process_word2vec_sen_vec_mode = "avg"

In [11]:
def norm_wmd(s1, s2):
    global word2vec_model_norm
    dis = np.nan_to_num(word2vec_model_norm.wmdistance(s1, s2))
    return dis if dis<100 else 100

In [12]:
def wmd(s1, s2):
    global word2vec_model
    dis = np.nan_to_num(word2vec_model.wmdistance(s1, s2))
    return dis if dis<100 else 100

In [13]:
def run_word2vec_sen_vec_wmd(df, ngram, prefix, name):
    q_list = df['query'].apply(ngram)
    t_list = df['title'].apply(ngram)
    df['%s_wmd%s' % (prefix, name)] = cal_sim(q_list, t_list, wmd)
    df['%s_normWmd%s' % (prefix, name)] = cal_sim(q_list, t_list, norm_wmd)
    del q_list, t_list
    gc.collect()
    return df

In [14]:
def run_word2vec_sen_vec_sk(df, func, q_sen_vec, t_sen_vec, prefix, name):
    df['%s_q%s' % (prefix, name)] = list(map(func, q_sen_vec))
    df['%s_t%s' % (prefix, name)] = list(map(func, t_sen_vec))
    df['%s_%s-diff'%(prefix, name)] = list(map(lambda a, b: abs(a-b), df['%s_q%s' % (prefix, name)], df['%s_t%s' % (prefix, name)]))
    df['%s_%s-max'%(prefix, name)] = list(map(lambda a, b: max(a, b), df['%s_q%s' % (prefix, name)], df['%s_t%s' % (prefix, name)]))
    df['%s_%s-min'%(prefix, name)] = list(map(lambda a, b: min(a, b), df['%s_q%s' % (prefix, name)], df['%s_t%s' % (prefix, name)]))
    df['%s_%s-avg'%(prefix, name)] = list(map(lambda a, b: (a+b)/2, df['%s_q%s' % (prefix, name)], df['%s_t%s' % (prefix, name)]))
    return df

In [15]:
def process_word2vec_sen_vec_other(df, prefix, savefile):
    
    global word2vec_model_norm, word2vec_model_file
    word2vec_model_norm = KeyedVectors.load(word2vec_model_file)
    word2vec_model_norm.init_sims(replace=True)
    
    # with Timer("cal uni-wmd dis"):
    #     df = run_word2vec_sen_vec_wmd(df, ngram_utils.unigrams, prefix, "Unigrams")
    # with Timer("cal bi-wmd dis"):
    #     df = run_word2vec_sen_vec_wmd(df, ngram_utils.bigrams, prefix, "Bigrams")
        
    with Timer("cal sent vec"):
        global process_word2vec_sen_vec_mode
        if process_word2vec_sen_vec_mode == "avg":
            q_sen_vec = list(map(sent2vec, df["query"]))
            t_sen_vec = list(map(sent2vec, df["title"]))
        elif process_word2vec_sen_vec_mode == "tfidf":
            q_sen_vec = list(map(sent2vec1, df["query"]))
            t_sen_vec = list(map(sent2vec1, df["title"]))
    
    with Timer("cal skew"):
        df = run_word2vec_sen_vec_sk(df, skew, q_sen_vec, t_sen_vec, prefix, "Skew")
    with Timer("cal kurtosis"):
        df = run_word2vec_sen_vec_sk(df, kurtosis, q_sen_vec, t_sen_vec, prefix, "Kurtosis")
        
    del q_sen_vec, t_sen_vec
    gc.collect()
    
    return df

In [19]:
# 提取训练数据特征
if regen_word2vec_sen_vec_other:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, 
        "senVec{}SK".format(process_word2vec_sen_vec_mode), 
        process_func=process_word2vec_sen_vec_other, names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, 
        process_chunkly=True, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], 
        drop_last_cols=['query', 'title'])
# Finished 'extract senVecavgSim feature' block, time used:4160.86s.

----->Started 'extract senVecavgSK feature' block...
----->Started 'process chunk' block...
Mem. usage decreased to 15.26 Mb (0.0% reduction)
----->Started 'cal sent vec' block...
----->Finished 'cal sent vec' block, time used:202.48s.
----->Started 'cal skew' block...
----->Finished 'cal skew' block, time used:548.07s.
----->Started 'cal kurtosis' block...
----->Finished 'cal kurtosis' block, time used:406.1s.
saved senVecavgSK feature to ./stage1/output/word2vec/online_trainWord2vec_senVecavgSK.csv.gz
----->Finished 'process chunk' block, time used:1233.19s.
----->Started 'process chunk' block...
Mem. usage decreased to 15.26 Mb (0.0% reduction)
----->Started 'cal sent vec' block...
----->Finished 'cal sent vec' block, time used:203.53s.
----->Started 'cal skew' block...
----->Finished 'cal skew' block, time used:548.57s.
----->Started 'cal kurtosis' block...
----->Finished 'cal kurtosis' block, time used:414.55s.
saved senVecavgSK feature to ./stage1/output/word2vec/online_trainWord

## Magic特征

In [12]:
feature_save_dir = base_feature_save_dir + "magic/"
train_feature_prefix = base_prefix + "trainMagic"
test_feature_prefix = base_prefix + "testMagic"
! ls -alh ./stage1/output/magic/
! rm ./stage1/output/magic/online_testMagic_diffSetSimTrigramsTitle.npy

total 7.4G
drwxr-xr-x 2 kesci root  4.0K Jun 30 01:44 .
drwxr-xr-x 7 kesci root  4.0K Jun 19 09:08 ..
-rw-r--r-- 1 kesci users 5.8K Jun 26 13:43 debug_trainMagic_ctrRateGlobal.csv.gz
-rw-r--r-- 1 kesci users 122K Jun 26 13:43 debug_trainMagic_ctrRateLocal.csv.gz
-rw-r--r-- 1 kesci users  31M Jun 27 12:19 debug_trainMagic_diffSetLenCharsBicharsTitle.npy
-rw-r--r-- 1 kesci users 2.7M Jun 27 12:26 debug_trainMagic_diffSetLenChars.csv.gz
-rw-r--r-- 1 kesci users 6.0M Jun 27 12:18 debug_trainMagic_diffSetLenCharsUnicharsTitle.npy
-rw-r--r-- 1 kesci users 970K Jun 26 13:19 debug_trainMagic_diffSetLen.csv.gz
-rw-r--r-- 1 kesci users 448K Jun 27 12:11 debug_trainMagic_diffSetLenUnichars.csv.gz
-rw-r--r-- 1 kesci users 6.0M Jun 27 12:11 debug_trainMagic_diffSetLenUnicharsUnicharsTitle.npy
-rw-r--r-- 1 kesci users  89M Jun 24 15:16 debug_trainMagic_diffSetSimBigramsTitle.npy
-rw-r--r-- 1 kesci users  31M Jun 27 12:16 debug_trainMagic_diffSetSimCharsBicharsTitle.npy
-rw-r--r-- 1 kesci users 3.0M 

In [20]:
tmp = np.load("./stage1/output/magic/debug_trainMagic_diffSetSimBigramsTitle.npy", allow_pickle=True)
print(tmp[:2])

[list(['3567_31451', '59_193', '663_82', '66324_32684', '27_37136', '31451_27', '43993_59', '37136_663', '288_33', '87_87', '18225_87', '82_18225', '33_43993', '1017_3567', '27_288'])
 list(['660_12875', '31_4126', '10964_799', '147_31', '9669_15457', '27_660', '12875_9669', '4126_15', '14514_27', '799_14514'])]


In [21]:
tmp = np.load("./stage1/output/magic/debug_trainMagic_diffSetSimUnigramsTitle.npy", allow_pickle=True)
print(tmp[:2])

[list(['33', '37136', '1017', '59', '87', '663', '18225', '3567'])
 list(['15457', '4126', '9669', '799', '660', '12875', '147', '14514'])]


### 提取query频率

In [32]:
regen_query_freq = False
train_query_freq = train_feature_prefix + "_queryFreq"
train_freq_save_file = feature_save_dir+train_query_freq+".csv.gz"
if not OFFLINE:
    test_query_freq = test_feature_prefix + "_queryFreq"
    test_freq_save_file = feature_save_dir+test_query_freq+".csv.gz"

In [29]:
def ExtractFreq(sourcefile, savefile, names, dtype, group_by, sort_col, feature_name):
    df = ReadCSV(sourcefile, names=names, dtype=dtype, iterator=False)
    grouped = df.groupby(group_by, as_index=False)[sort_col].count()
    res = np.concatenate(list(map(lambda x: [x]*x, grouped[sort_col])))
    tmp = pd.DataFrame({
        feature_name: res
    })
    tmp.to_csv(savefile, compression="gzip", index=None)
    print("Feature saved to {}".format(savefile))
    del df, grouped, res, tmp
    gc.collect()

In [30]:
if regen_query_freq:
    ExtractFreq(train_file, 
        train_freq_save_file, 
        ORI_TRAIN_NAMES, 
        ORI_TRAIN_DTYPE,
        "query_id",
        "query_title_id",
        "query_freq")

Feature saved to ./stage1/output/magic/debug_trainMagic_queryFreq.csv.gz


In [31]:
if regen_query_freq and not OFFLINE:
    ExtractFreq(test_file, 
        test_freq_save_file, 
        ORI_TEST_NAMES, 
        ORI_TEST_DTYPE,
        "query_id",
        "query_title_id",
        "query_freq")

In [96]:
# AnalysisCSV("./stage1/output/magic/debug_trainMagic_queryFreq.csv.gz", print_rows=20)

In [25]:
# AnalysisCSV("./stage1/output/magic/debug_trainMagic_queryFreq.csv.gz")

### 提取差集特征

In [33]:
regen_diff_sim = True
similar_arr = [
    dist_utils.dice_ratio,
    dist_utils.jaccard_ratio,
    dist_utils.edit_seq_ratio,
    dist_utils.edit_set_ratio,
]

In [34]:
def findDiff(arr, ngram):
    set_list = list(map(lambda x: set(ngram(x)), arr))
    res = []
    for i, _ in enumerate(set_list):
        result = _
        for __ in set_list:
            if _ != __:
                result = result.difference(__)
        res.append(list(result))
    return res

In [35]:
def GetDiffSet(query_id, title, ngram):
    tmp, res = [], []
    for i in range(len(query_id)):
      if i == 0:
        tmp.append(title[i])
      else:
        if query_id[i] == query_id[i-1]:
          tmp.append(title[i])
        else:
          res.append(findDiff(tmp, ngram))
          tmp = [title[i]]
    res.append(findDiff(tmp, ngram))
    result = []
    for _ in res:
        result.extend(_)
    return result

In [36]:
def run_diff_set_sim(df, ngram, prefix, savefile, name):
    with Timer("cal diff set"):
        t_savefile = savefile + name +"Title.npy"
        q_list = df['query'].apply(ngram)
        if os.path.exists(t_savefile):
            t_list = np.load(t_savefile, allow_pickle=True)
        else:
            t_list = GetDiffSet(df["query_id"], df["title"], ngram)
            np.save(t_savefile, t_list) # 存储处理后的title列表
            print("File saved to {}".format(t_savefile))
    ## 长度特征
    with Timer("extract similarity"):
        for _ in similar_arr:
            name = _.__name__
            with Timer("cal {} sim".format(name)):
                df["{}_diffSetSim{}".format(prefix, name)] = cal_sim(q_list, t_list, _)
    del q_list, t_list
    gc.collect()
    return df
    
def process_diff_set_sim(df, prefix, savefile):
    # df = run_diff_set_sim(df, ngram_utils.unichars, '%s_%s' % (prefix, 'Unichars'), savefile, "Unichars")
    # df = run_diff_set_sim(df, ngram_utils.bichars, '%s_%s' % (prefix, 'Bichars'), savefile, "Bichars")
    df = run_diff_set_sim(df, ngram_utils.unigrams, '%s_%s' % (prefix, 'unigrams'), savefile, "Unigrams")
    df = run_diff_set_sim(df, ngram_utils.bigrams, '%s_%s' % (prefix, 'bigrams'), savefile, "Bigrams")
    df = run_diff_set_sim(df, ngram_utils.trigrams, '%s_%s' % (prefix, 'trigrams'), savefile, "Trigrams")
    return df

In [37]:
# 提取训练数据特征
if regen_diff_sim:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "diffSetSim", 
        process_func=process_diff_set_sim, 
        names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['label', 'query_title_id'], drop_last_cols=['query_id', 'query', 'title'])

----->Started 'extract diffSetSim feature' block...
Mem. usage decreased to  9.54 Mb (0.0% reduction)
----->Started 'cal diff set' block...
----->Finished 'cal diff set' block, time used:3.54s.
----->Started 'extract similarity' block...
----->Started 'cal dice_ratio sim' block...
----->Finished 'cal dice_ratio sim' block, time used:1.82s.
----->Started 'cal jaccard_ratio sim' block...
----->Finished 'cal jaccard_ratio sim' block, time used:1.6s.
----->Started 'cal edit_seq_ratio sim' block...
----->Finished 'cal edit_seq_ratio sim' block, time used:3.21s.
----->Started 'cal edit_set_ratio sim' block...
----->Finished 'cal edit_set_ratio sim' block, time used:3.95s.
----->Finished 'extract similarity' block, time used:10.58s.
----->Started 'cal diff set' block...
----->Finished 'cal diff set' block, time used:5.08s.
----->Started 'extract similarity' block...
----->Started 'cal dice_ratio sim' block...
----->Finished 'cal dice_ratio sim' block, time used:1.74s.
----->Started 'cal jacca

In [80]:
# AnalysisCSV("./stage1/output/magic/debug_trainMagic_diffSetSim1.csv.gz")

In [39]:
# 提取训练数据特征
if regen_diff_sim:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, "diffSetSim", 
        process_func=process_diff_set_sim, 
        names=ORI_TEST_NAMES, dtype=ORI_TEST_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_title_id'], drop_last_cols=['query_id', 'query', 'title'])

----->Started 'extract diffSetSim feature' block...
Mem. usage decreased to 95.37 Mb (0.0% reduction)
----->Started 'cal diff set' block...
----->Finished 'cal diff set' block, time used:22.76s.
----->Started 'extract similarity' block...
----->Started 'cal dice_ratio sim' block...
----->Finished 'cal dice_ratio sim' block, time used:12.34s.
----->Started 'cal jaccard_ratio sim' block...
----->Finished 'cal jaccard_ratio sim' block, time used:11.17s.
----->Started 'cal edit_seq_ratio sim' block...
----->Finished 'cal edit_seq_ratio sim' block, time used:24.96s.
----->Started 'cal edit_set_ratio sim' block...
----->Finished 'cal edit_set_ratio sim' block, time used:29.84s.
----->Finished 'extract similarity' block, time used:78.31s.
----->Started 'cal diff set' block...
----->Finished 'cal diff set' block, time used:40.83s.
----->Started 'extract similarity' block...
----->Started 'cal dice_ratio sim' block...
----->Finished 'cal dice_ratio sim' block, time used:15.5s.
----->Started 'ca

In [1]:
# AnalysisCSV("./stage1/output/magic/online_trainMagic_diffSetSim.csv.gz")

### 差集的tfidf

In [52]:
from np_utils import try_divide
tfidf_model_file = "./stage1/output/vector_space/debug_tfidf_model.bin"

In [35]:
with open(tfidf_model_file, "rb") as ff:
    tfidf_model = pickle.load(ff)
    tfidf_vocab = tfidf_model.vocabulary_
    tfidf_idf = tfidf_model.idf_

In [48]:
def get_uni_sum_tfidf(ngrams):
    l = len(ngrams)
    return sum([tfidf_idf[tfidf_vocab[_]]/l for _ in ngrams if _ in tfidf_vocab])

In [50]:
def run_diff_set_tfidf_len(df, ngram, prefix, savefile, name):
    with Timer("cal diff set"):
        t_savefile = savefile + name +"Title.npy"
        t_savefile = t_savefile.replace("TfidfLen", "Sim")
        if os.path.exists(t_savefile):
            t_list = np.load(t_savefile, allow_pickle=True)
        else:
            pass
    ## 长度特征
    with Timer("extract similarity"):
        t_vec_len = list(map(get_uni_sum_tfidf, t_list))
        q_vec_len = np.array(tfidf_model.transform(df["query"]).sum(1)).squeeze()
        df['%s_TLen1' % prefix] = t_vec_len
        df['%s_QTLenRatio1'%prefix] = list(map(lambda a, b: try_divide(a, b), q_vec_len, t_vec_len))
        df['%s_TQLenRatio1'%prefix] = list(map(lambda a, b: try_divide(b, a), q_vec_len, t_vec_len))
        df['%s_QTDiff1'%prefix] = list(map(lambda a, b: abs(a-b), q_vec_len, t_vec_len))
        df['%s_QTMax1'%prefix] = list(map(lambda a, b: max(a, b), q_vec_len, t_vec_len))
        df['%s_QTMin1'%prefix] = list(map(lambda a, b: min(a, b), q_vec_len, t_vec_len))
        df['%s_QTAvg1'%prefix] = list(map(lambda a, b: (a+b)/2, q_vec_len, t_vec_len))
        df['%s_QTMulti1'%prefix] = list(map(lambda a, b: a*b, q_vec_len, t_vec_len))
    del t_list, t_vec_len, q_vec_len
    gc.collect()
    return df
    
def process_diff_set_tfidf_len(df, prefix, savefile):
    # df = run_diff_set_sim(df, ngram_utils.unichars, '%s_%s' % (prefix, 'Unichars'), savefile, "Unichars")
    # df = run_diff_set_sim(df, ngram_utils.bichars, '%s_%s' % (prefix, 'Bichars'), savefile, "Bichars")
    df = run_diff_set_tfidf_len(df, ngram_utils.unigrams, '%s_%s' % (prefix, 'unigrams'), savefile, "Unigrams")
    # df = run_diff_set_sim(df, ngram_utils.bigrams, '%s_%s' % (prefix, 'bigrams'), savefile, "Bigrams")
    return df

In [53]:
# 提取训练数据特征
ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "diffSetTfidfLen", 
    process_func=process_diff_set_tfidf_len, 
    names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
    drop_first_cols=['label', 'query_title_id'], drop_last_cols=['query_id', 'query', 'title'])

----->Started 'extract diffSetTfidfLen feature' block...
Mem. usage decreased to  9.54 Mb (0.0% reduction)
----->Started 'cal diff set' block...
----->Finished 'cal diff set' block, time used:2.95s.
----->Started 'extract similarity' block...
----->Finished 'extract similarity' block, time used:21.96s.
saved diffSetTfidfLen feature to ./stage1/output/magic/debug_trainMagic_diffSetTfidfLen.csv.gz
----->Finished 'extract diffSetTfidfLen feature' block, time used:51.98s.


### 点击率特征

In [38]:
regen_ctr_rate = True
stat = {}

In [39]:
def ExtractGlobalCTR(df, prefix, savefile):
    # 全局点击率
    tmp = df["label"].value_counts()
    global_click_rate = tmp[1] / (tmp[0] + tmp[1])
    feature_name = "{}_globalCTR".format(prefix)
    df[feature_name] = [global_click_rate] * df.shape[0]
    if not OFFLINE:
        prefix = prefix.replace("train", "test")
        feature_name = feature_name.replace("train", "test")
        savefile = savefile.replace("train", "test")+".csv.gz"
        test_df = pd.DataFrame({
            feature_name: [global_click_rate] * 5000000
        }).to_csv(savefile, index=None, compression="gzip")
        print("Feature saved to {}".format(savefile))
        del test_df
    del tmp
    gc.collect()
    return df

In [40]:
if regen_ctr_rate:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "ctrRateGlobal", 
    ExtractGlobalCTR, names=ORI_TRAIN_NAMES,
    dtype=None, process_chunkly=False, 
    drop_first_cols=['query_title_id', 'query', 'title'], drop_last_cols=['query_id', 'label'])

----->Started 'extract ctrRateGlobal feature' block...
Mem. usage decreased to  2.38 Mb (68.7% reduction)
saved ctrRateGlobal feature to ./stage1/output/magic/debug_trainMagic_ctrRateGlobal.csv.gz
----->Finished 'extract ctrRateGlobal feature' block, time used:4.06s.


In [41]:
def ExtractLocalCTR(df, prefix, savefile):
    global stat
    grouped = df.groupby("query_id", as_index=False)
    count = grouped.size().values
    if not stat:
        label = grouped["label"].sum()["label"].values.astype("int32")
        for _ in range(len(count)):
            item_len = count[_]
            if item_len not in stat:
                stat[item_len] = [1, label[_]]
            else:
                stat[item_len][0] += 1
                stat[item_len][1] += label[_]
        del label
        print(stat)
    res = [[ stat[_][1] / (_ * stat[_][0])]*_ for _ in count]
    result = []
    for _ in res:
        result.extend(_)
    df["{}_localCTR".format(prefix)] = result
    del res, grouped
    gc.collect()
    return df

In [42]:
if regen_ctr_rate:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "ctrRateLocal", 
    ExtractLocalCTR, names=ORI_TRAIN_NAMES,
    dtype=None, process_chunkly=False, 
    drop_first_cols=['query_title_id', 'query', 'title'], drop_last_cols=['query_id', 'label'])

----->Started 'extract ctrRateLocal feature' block...
Mem. usage decreased to  2.38 Mb (68.7% reduction)
{11: [1834, 4011], 17: [658, 1727], 13: [1236, 2857], 3: [33728, 44915], 20: [4540, 12425], 4: [5026, 7386], 5: [4753, 7609], 7: [4878, 8538], 6: [4425, 7450], 8: [4544, 8524], 9: [2336, 4671], 12: [1523, 3377], 14: [1023, 2473], 19: [517, 1450], 16: [795, 2057], 15: [886, 2202], 10: [1983, 4125], 18: [577, 1578]}
saved ctrRateLocal feature to ./stage1/output/magic/debug_trainMagic_ctrRateLocal.csv.gz
----->Finished 'extract ctrRateLocal feature' block, time used:5.16s.


In [43]:
if not OFFLINE and regen_ctr_rate:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, "ctrRateLocal", 
    ExtractLocalCRT, names=ORI_TEST_NAMES,
    dtype=None, process_chunkly=False, 
    drop_first_cols=['query_title_id', 'query', 'title'], drop_last_cols=['query_id'])

### diffSetLen

In [16]:
def run_diff_set_len(df, ngram, prefix, savefile, name):
    with Timer("cal diff set"):
        t_savefile = savefile + name +"Title.npy"
        t_savefile = t_savefile.replace("Len", "Sim")
        q_list = df['query'].apply(ngram)
        print(t_savefile)
        if os.path.exists(t_savefile):
            t_list = np.load(t_savefile, allow_pickle=True)
        else:
            pass
            # t_list = GetDiffSet(df["query_id"], df["title"], ngram)
            # np.save(t_savefile, t_list) # 存储处理后的title列表
            # print("File saved to {}".format(t_savefile))
    ## 长度特征
    with Timer("extract length rel"):
        q_len = np.vectorize(len)(q_list)
        t_len = np.vectorize(len)(t_list)
        df['%s_difft-len' % prefix] = t_len
        df['%s_diffqt-len-radio'%prefix] = list(map(lambda a, b: a/b, q_len, t_len))
        df['%s_difftq-len-radio'%prefix] = list(map(lambda a, b: b/a, q_len, t_len))
        df['%s_diffqt-diff'%prefix] = list(map(lambda a, b: abs(a-b), q_len, t_len))
        df['%s_diffqt-max'%prefix] = list(map(lambda a, b: max(a, b), q_len, t_len))
        df['%s_diffqt-min'%prefix] = list(map(lambda a, b: min(a, b), q_len, t_len))
        df['%s_diffqt-avg'%prefix] = list(map(lambda a, b: (a+b)/2, q_len, t_len))
    del q_list, t_list, q_len
    gc.collect()
    return df
    
def process_diff_set_len(df, prefix, savefile):
    df = run_diff_set_sim(df, ngram_utils.unichars, '%s_%s' % (prefix, 'Unichars'), savefile, "Unichars")
    df = run_diff_set_sim(df, ngram_utils.bichars, '%s_%s' % (prefix, 'Bichars'), savefile, "Bichars")
    # df = run_diff_set_len(df, ngram_utils.unigrams, '%s_%s' % (prefix, 'unigrams'), savefile, "Unigrams")
    # df = run_diff_set_len(df, ngram_utils.bigrams, '%s_%s' % (prefix, 'bigrams'), savefile, "Bigrams")
    # df = run_diff_set_len(df, ngram_utils.trigrams, '%s_%s' % (prefix, 'trigrams'), savefile, "Trigrams")
    return df

In [17]:
ExtractFeature(train_file, feature_save_dir, train_feature_prefix, "diffSetLen", 
    process_func=process_diff_set_len, 
    names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
    drop_first_cols=['label', 'query_title_id'], drop_last_cols=['query_id', 'query', 'title'])

----->Started 'extract diffSetLen feature' block...
Mem. usage decreased to 190.73 Mb (0.0% reduction)
----->Started 'cal diff set' block...
./stage1/output/magic/online_trainMagic_diffSetSimUnigramsTitle.npy
----->Finished 'cal diff set' block, time used:44.36s.
----->Started 'extract length rel' block...


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in long_scalars


----->Finished 'extract length rel' block, time used:52.39s.
saved diffSetLen feature to ./stage1/output/magic/online_trainMagic_diffSetLen.csv.gz
----->Finished 'extract diffSetLen feature' block, time used:265.21s.


In [18]:
ExtractFeature(test_file, feature_save_dir, test_feature_prefix, "diffSetLen", 
    process_func=process_diff_set_len, 
    names=ORI_TEST_NAMES, dtype=ORI_TEST_DTYPE, process_chunkly=False, chunk_size=CHUNK_SIZE, 
    drop_first_cols=['label', 'query_title_id'], drop_last_cols=['query_id', 'query', 'title'])

----->Started 'extract diffSetLen feature' block...
Mem. usage decreased to 95.37 Mb (0.0% reduction)
----->Started 'cal diff set' block...
./stage1/output/magic/online_testMagic_diffSetSimUnigramsTitle.npy
----->Finished 'cal diff set' block, time used:24.19s.
----->Started 'extract length rel' block...


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in long_scalars


----->Finished 'extract length rel' block, time used:30.42s.
saved diffSetLen feature to ./stage1/output/magic/online_testMagic_diffSetLen.csv.gz
----->Finished 'extract diffSetLen feature' block, time used:141.74s.


In [ ]:
! ps aux